## 1. Обучаем word2vec

In [97]:
import gzip
import numpy as np
import os
import random
from dataclasses import dataclass
from typing import Iterator, List

@dataclass
class Text:
    label: str
    title: str
    text: str


def read_texts(fn: str) -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))

texts = list(read_texts("../data/news.txt.gz"))

In [98]:
import nltk
import string
from nltk.corpus import stopwords

stopwords_ru = set(stopwords.words('russian'))
# Разбиение текста на слова             
def tokenize_text(text: str) -> List[str]:
    text = text.lower()
    words = nltk.WordPunctTokenizer().tokenize(text)
    return words

# Удаление знаков припенания и стоп-слов
def normalize_text(words: str) -> str:
    filterd_words = [w for w in words if all(c not in string.punctuation for c in w) ]
    words = [word for word in filterd_words if word not in stopwords_ru]
    return words

In [99]:
#nltk.download('stopwords')

In [100]:
normalize_text(tokenize_text(texts[0].text))

['парусная',
 'гонка',
 'giraglia',
 'rolex',
 'cup',
 'пройдет',
 'средиземном',
 'море',
 '64',
 'й',
 'победители',
 'соревнования',
 'проводимого',
 '1953',
 'года',
 'yacht',
 'club',
 'italiano',
 'помимо',
 'других',
 'призов',
 'традиционно',
 'получают',
 'подарок',
 'часы',
 'швейцарского',
 'бренда',
 'rolex',
 'сообщается',
 'пресс',
 'релизе',
 'поступившем',
 'редакцию',
 '«',
 'ленты',
 'ру',
 '»',
 'среду',
 '8',
 'мая',
 'rolex',
 'yacht',
 'master',
 '40',
 'фото',
 'пресс',
 'служба',
 'mercury',
 'соревнования',
 'будут',
 'проходить',
 '10',
 '18',
 'июня',
 'первый',
 'этап',
 'ночной',
 'переход',
 'сан',
 'ремо',
 'сен',
 'тропе',
 '10',
 '11',
 'июня',
 'дистанция',
 '50',
 'морских',
 'миль',
 '—',
 'около',
 '90',
 'километров',
 'второй',
 'этап',
 'серия',
 'прибрежных',
 'гонок',
 'бухте',
 'сен',
 'тропе',
 '11',
 '14',
 'июня',
 'финальный',
 'этап',
 'пройдет',
 '15',
 '18',
 'июня',
 'оффшорная',
 'гонка',
 'маршруту',
 'сен',
 'тропе',
 '—',
 'генуя',

In [101]:
WORDVEC_LEN = 300

In [102]:
SEED = 0
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

In [103]:
from gensim.models import Word2Vec

## Обучение word2vec
# каждый текст - набор слов через пробел
sentences = [normalize_text(tokenize_text(text.text)) for text in texts]
# обучаем w2v (при workers > 1 теряется детерменизм)
w2v = Word2Vec(sentences, vector_size=WORDVEC_LEN, seed=SEED, workers=1)

# сохраняем модель
#w2v.wv.save_word2vec_format('w2v_vectors.bin')

In [104]:
w2v.wv.most_similar("новости")

[('итар', 0.8214165568351746),
 ('антидопинговое', 0.7923288345336914),
 ('агентство', 0.7847864031791687),
 ('всемирное', 0.7839740514755249),
 ('источник', 0.7794643640518188),
 ('ссылкой', 0.7669432163238525),
 ('тасс', 0.7635668516159058),
 ('прайм', 0.7425318956375122),
 ('белта', 0.7405049204826355),
 ('сообщило', 0.7379317879676819)]

In [105]:
# пример
w2v.wv.most_similar("канал")

[('тнт', 0.9213761687278748),
 ('телеканале', 0.9120844602584839),
 ('бердыев', 0.8950342535972595),
 ('ефремов', 0.8912442326545715),
 ('стратеджист', 0.8906193375587463),
 ('роман', 0.8904144167900085),
 ('обязала', 0.8882070779800415),
 ('гастролями', 0.8871371150016785),
 ('холле', 0.8869787454605103),
 ('показ', 0.8864108324050903)]

### Train test split

In [106]:
from sklearn.model_selection import train_test_split

X = [text.text for text in texts]
y = [text.label for text in texts]

raw_train, raw_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=SEED)

## 2. Получим эмбеддинги текстов, усреднив векторные представления слов:

In [107]:
import numpy as np

In [108]:
def tok(text):
    return normalize_text(tokenize_text(text))

In [109]:
def get_doc_emb_avg(doc):
    res = np.zeros(WORDVEC_LEN)
    for word in tok(doc):
        if word in w2v.wv.key_to_index.keys(): # если слова нет в словаре, пока игнорируем (эмбеддинг=0)
            res += w2v.wv.get_vector(word)
    return res / len(doc)

In [110]:
X_train = [get_doc_emb_avg(doc) for doc in raw_train]
X_test = [get_doc_emb_avg(doc) for doc in raw_test]

## 3. Обучим Multinominal Naive Bayes:

In [111]:
topics = {'science':0, 'style':1, 'culture':2, 'life':3, 'economics':4, 'business':5, 'travel':6, 'forces':7, 'media':8, 'sport':9}

In [112]:
y_train = list(map(topics.get, y_train))
y_test = list(map(topics.get, y_test))
y_train[:5]

[9, 9, 0, 7, 4]

In [113]:
param_grid = [{'alpha':0.0001*np.arange(1,11)}]

In [114]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.naive_bayes import MultinomialNB   
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler #fixed import

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [115]:
# make custom scorer so we pass multi_class='ovr' to GridSearch scorer
multi_roc = make_scorer(roc_auc_score, average='weighted', multi_class='ovr', needs_proba=True)

In [116]:
clf = MultinomialNB()
search = GridSearchCV(clf, param_grid, cv=5, scoring=multi_roc)
search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid=[{'alpha': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009, 0.001 ])}],
             scoring=make_scorer(roc_auc_score, needs_proba=True, average=weighted, multi_class=ovr))

In [117]:
best_clf = search.best_estimator_
best_clf.fit(X_train,y_train)

MultinomialNB(alpha=0.0009000000000000001)

In [118]:
roc_auc_score(y_test,best_clf.predict_proba(X_test), multi_class='ovr')

0.8812412663092953

## 4. Придумаем другой способ  
Можно для каждого слова текста получить его эмбеддинг из Tfidf с помощью TfidfVectorizer из sklearn.  
Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf.

In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tok)
X_tfidf = vectorizer.fit_transform(X)

c:\Users\Laptop_user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


И уменьшим размерность до длины вектора word2vec:

In [120]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=WORDVEC_LEN, random_state=SEED)
X_dense = svd.fit_transform(X_tfidf.T)
X_dense

array([[ 4.98041709e-01, -8.43084359e-02,  5.82440799e-01, ...,
        -2.22698738e-04,  2.15129015e-03, -5.81086267e-04],
       [ 1.68012076e-01, -3.57053609e-02,  2.67001606e-02, ...,
        -4.69217029e-02, -1.62820510e-02,  2.37969532e-02],
       [ 1.08247770e-02, -2.59344276e-03, -9.04583249e-03, ...,
         9.05266406e-04, -9.87069197e-03,  3.22248969e-03],
       ...,
       [ 1.60523019e-04, -1.37969400e-04, -4.94428330e-04, ...,
        -9.42718152e-04, -2.49868798e-04,  7.22463158e-04],
       [ 2.31620735e-04, -2.72414259e-04, -3.95700300e-04, ...,
         1.07629597e-04,  8.43313120e-04,  1.07053512e-03],
       [ 5.74321131e-04, -2.14859552e-04, -5.99974139e-04, ...,
         3.07682708e-04, -3.60926043e-04,  7.71627451e-05]])

In [121]:
X_dense.shape

(149707, 300)

In [122]:
len(vectorizer.get_feature_names_out())

149707

In [123]:
keys = vectorizer.get_feature_names_out()
emb_dict = {keys[i]: X_dense[i] for i in range(len(keys))}

In [124]:
def get_doc_emb_with_tfidf(doc):
    res = np.zeros(WORDVEC_LEN)
    for word in tok(doc):
        res += np.array(emb_dict[word.lower()])
        if word.lower() in w2v.wv.key_to_index.keys(): # если слова нет в словаре, пока игнорируем (эмбеддинг=0)
            res += w2v.wv.get_vector(word)
    return res / len(doc)

In [125]:
X_train = [get_doc_emb_with_tfidf(doc) for doc in raw_train]
X_test = [get_doc_emb_with_tfidf(doc) for doc in raw_test]

In [126]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
clf = MultinomialNB()
search = GridSearchCV(clf, param_grid, cv=5, scoring=multi_roc)
search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid=[{'alpha': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009, 0.001 ])}],
             scoring=make_scorer(roc_auc_score, needs_proba=True, average=weighted, multi_class=ovr))

In [127]:
best_clf = search.best_estimator_
best_clf.fit(X_train,y_train)

MultinomialNB(alpha=0.0006000000000000001)

In [128]:
roc_auc_score(y_test,best_clf.predict_proba(X_test), multi_class='ovr')

0.8823532003880175

Из-за того, что word2vec и так учитывает контекст слов, значительного прироста в качестве получить не удалось